# 20_ordinal_nonlikert — 비-Likert 순서형

In [ ]:
%run 00_config.ipynb
import pandas as pd, numpy as np

def reverse(series, max_scale): return max_scale + 1 - series
def wprop(series, w):
    s = series.dropna()
    w = w.loc[s.index]
    tot = w.sum()
    if tot<=0: return pd.Series(dtype=float)
    return (w.groupby(s).sum()/tot*100).sort_index()

df = pd.read_excel(DATA_PATH)
w = choose_weight(df)

onl = df.copy()[ONL_VARS]

# DK 처리
for col, dk in ONL_DK.items():
    if col in onl.columns:
        onl[col] = onl[col].where(onl[col] != dk, pd.NA)

# 역코딩
for col, mx in ONL_REVERSE.items():
    if col in onl.columns:
        onl[col] = reverse(onl[col], mx)

# 파생
onl["q12_pos"] = onl["q12"] if "q12" in onl.columns else np.nan
if "q13" in onl.columns:
    onl["q13_bin"] = onl["q13"].map(lambda x: 1 if x==1 else (0 if pd.notna(x) else pd.NA))

# 요약
rows = []
for col in ["q4","q8","q12","DM4","DM10"]:
    if col not in onl.columns: continue
    ws = wprop(onl[col], w)
    us = (onl[col].value_counts(dropna=True, normalize=True)*100).round(2)
    for lvl, pct in ws.items():
        rows.append({"variable": col, "level": lvl, "weighted_%": round(pct,2)})
    for lvl, pct in us.items():
        rows.append({"variable": col, "level": lvl, "unweighted_%": pct})

summary = pd.DataFrame(rows).groupby(["variable","level"], as_index=False).sum(numeric_only=True)\
                            .sort_values(["variable","level"])

summary.to_csv(OUT_DIR/"ordinal_nonlikert_summary.csv", index=False, encoding="utf-8-sig")
onl.to_csv(OUT_DIR/"ordinal_nonlikert_encoded.csv", index=False, encoding="utf-8-sig")
summary.head(20)
